# Tracer flow (single-phase, two-component flow)

## Introduction

This tutorial demonstrates how to set up a single-phase, 2-component model using water
and a tracer. The assumptions on the tracer are that it has no effects on the fluid
properties or flow dynamics, hence a pure tracer. The spreading of the tracer component
throughout the domain is simulated using a hyperbolic transport equation, also referred
to as component mass balance for the tracer.

The full model is implemented in `porepy.examples.tracer_flow` and a step-by-step guide
of the implementation is given here.

## Mathematical model

To understand the required steps and classes, let's first recount the mathematical model.

### Single phase flow

This model aspect is covered in full detail in the respective [tutorial](./single_phase_flow.ipynb).
We recall here that the physics of single phase flow introduce one variable, pressure $p$,
and one governing equation, the (total) fluid mass balance:

$$
\dfrac{\partial}{\partial t}\left(\Phi \rho\right) - \nabla\cdot\left( \dfrac{\rho}{\mu} \mathbf{K} \nabla p\right) = 0.
$$

### Transport equation

Porepy utilizes the overall fraction formulation for multi-component fluids.
The mass corresponding to a component $i$ is represented by an overall fraction $z_i$.
One component can arbitrarily be chosen as the reference component and its overall fraction $z_r$
is computed by unity of fractions

$$
z_r = 1 - \sum_{i\neq r} z_i,
$$
i.e. it is a dependent variable. For the sake of our model, we set water as the reference component
and the only remaining quantity of interest is the tracer fraction $z$.

The mass in one phase is in general also split between the present components, leading to
the requirement to introduce partial fractions of a component $i$ in some phase $j$.
But since we assume single-phase flow, the fraction of mass of one component in the single phase
is due to conservation of mass equal to its overall fraction.

This allows us to write the tracer transport equation for the unknown $z$ in the following compact form:

$$
\dfrac{\partial}{\partial t}\left(\Phi \rho z\right) - \nabla\cdot\left( \dfrac{\rho z}{\mu} \mathbf{K} \nabla p\right) = 0.
$$

### Summary

In a simplified setting with only 1 phase and 2 components, the variables of interest are

- pressure $p$ and
- independent overall fraction $z$.

The governing equations are given by

- total mass balance (pressure equation) and
- one component mass balance (tracer transport equation).

The extension of this model to the mixed-dimensional setting is a technicality, which can be omitted here.
In short, the mass flux between subdomains (*interface darcy flux*) is seen as the flux of overall mass.
The mass flux of one component is modelled analogously to the balance equation:
The (total) interface darcy flux is scaled down with the fraction $z$.
The choice of $z$ on the interface is handled via Upwind-coupling.

## Building blocks of the PorePy model

We start by fetching the single-phase flow physics and classes required to extend the model to two components.
Note that the `SinglePhaseFlow` class already includes the pressure variable $p$, the (total) mass balance, and boundary and initial conditions for $p$.

In [1]:
import porepy as pp
from porepy.models.fluid_mass_balance import SinglePhaseFlow

# A mixin handling the introduction of required fractional variables for arbitrary mixtures
from porepy.compositional.compositional_mixins import CompositionalVariables

# Mixins handling BC and IC for fractional variables, and a mixin for the
# component mass balance equation for the tracer.
from porepy.models.compositional_flow import (
    BoundaryConditionsFractions,
    ComponentMassBalanceEquations,
    InitialConditionsFractions,
)

We define the two-component fluid now and use some provided values for water.
For single-phase flow, the only method we need to override is the one defining the
fluid components `get_components`.

This method is originally implemented in the `FluidMixin`, which is already part of the `SinglePhaseFlow`.
It will by default create a single, liquid-like phase.

> Note:
>
>   The water is chosen as the first component, which (also by default) will set is as the reference component.

In [2]:
from porepy.applications.material_values.fluid_values import water

class TracerFluid:
    """Setting up a 2-component fluid."""

    def get_components(self) -> list[pp.FluidComponent]:
        """Mixed in method defining water as the reference component and a simple
        tracer as the second component."""

        component_1 = pp.FluidComponent(name="water", **water)
        component_2 = pp.FluidComponent(name="tracer")
        return [component_1, component_2]

In [3]:
class ChannelWithFracture:
    pass

In [4]:
class TracerIC(InitialConditionsFractions):
    pass

class TracerBC(BoundaryConditionsFractions):
    pass

Finally, let's define the model set-up using above building blocks.

- The domain and the fluid are pure mixins, in the sense that they override some methods
  which are already part of `SinglePhaseFlow`.
- The `CompositionalVariables` mixin includes functionality to create the tracer fraction $z$.
- The `ComponentMassBalanceEquations` will add transport equations for each independent component, which is the tracer in our case.
- `TracerIC` and `TracerBC` inherit functionality to set IC and BC for our fraction $z$,
  but also contain pure mixed-in methods to override IC and BC for pressure. Those methods are originally part of `SinglePhaseFlow`.
  This is done to reduce inheritance tree and avoid troubles with Python's MRO resolution.
- Lastly, the `SinglePhaseFlow` is added, which contains also constitutive laws for the phase density and viscosity.
  We exploit the default implementation which uses heuristic functions and accesses the `compressibility` and `viscosity` of the water component.
  I.e., the tracer has no effect on the thermodynamic properties of the fluid.

In [5]:

class TracerFlowSetup(
    ChannelWithFracture,
    TracerFluid,
    CompositionalVariables,
    ComponentMassBalanceEquations,
    TracerBC,
    TracerIC,
    SinglePhaseFlow,
):
    """Complete set-up for tracer flow modelled as a single phase, 2-component flow
    problem."""

## What we have explored

1. We have shown how to create a 2-component fluid.
2. We have explored the relevant building blocks for a single-phase, 2-component isothermal flow model with heuristic thermodynamic properties.
3. The extension to multi-component fluids is trivial. Just add more components and respective IC & BC.
4. we have shown the parallels between the mathematical model and representations of individual aspects as PorePy classes. 